In [15]:
#import scipy.stats as ss
import matplotlib.pyplot as plt
import cupy as cp
import cupy.random as cr
from tqdm import tqdm
import time

In [22]:


def vec_particle_gen(vp,nParticles):

    # Generate particles with maxwell distribution              
    # vp is the most probable speed of the maxwell distribution 

    
    velocity_vec = (vp/cp.sqrt(2))*cr.uniform( size = (3,1,nParticles) )
    
    phase_vec = cr.uniform(  size = (1,1,nParticles) )*2*cp.pi
    
    amp_vec =  cr.normal( size = (3,1,nParticles) )
    
    normfac = 1/cp.sqrt(cp.sum(amp_vec[:,0,:]**2,axis=0))
    
    amp_vec[:,0,:] =  amp_vec[:,0,:]*cp.tile(normfac,(3,1))    

    
    return velocity_vec, amp_vec , phase_vec

def vector_random_walk(tX,omega,K,amp,phase):
    


    nParticles = omega.shape[2]
    nPoints = tX.shape[1]
    ret = cp.ndarray( (3,nPoints ) )
    
    phase_tensor = cp.tile(phase,(1,nPoints,1))
    amp_tensor = cp.tile(amp,(1,nPoints,1))
                     
    for i in [0,1,2]:          
        ret[i,:] = cp.sum( amp_tensor[i,:,:]*cp.sin(cp.outer(tX[0,:],omega[0,0,:]) 
                                  - cp.matmul(tX[1:,:].transpose(),K[:,0,:])
                                  + phase_tensor[:,0,:]), axis=1) 
    return ret




Statistics.

In [24]:
'''
Get the marginal distribution
'''
vp=1e-3
log10m = -22
m=10**log10m

t_corr = 2*cp.pi/m
l_dB = 2*cp.pi/m/vp
#ev2kpc = 3.0857e19 /1.973e-7


nPoints = 16

tx = cp.zeros((4,nPoints))
tlist= cp.linspace(0,1,nPoints+1)
tx[0,:] = tlist[:-1]*t_corr

nParticles = 1024
output=[]

nSimulation= 100
Amps = cp.ndarray((3,nSimulation))
Phis = cp.ndarray((3,nSimulation))


'''
Main:
'''
for i in tqdm(range(nSimulation)):
    
    v,a,p=vec_particle_gen(vp,nParticles)
    
    # Physical values   
    m = 10**log10m # eV
    K = m*v

    # From K to omega
    omega = cp.ndarray((1,nPoints,nParticles))
    omega[0,:,:] = cp.sqrt(cp.sum(K*K,axis=0)+m**2)
    
    #Retx(3,nPoints)
    Ret = vector_random_walk(tx,omega,K,a,p)
    
    #results(3,nSimulation)
    Amps[:,i] = 2 * cp.sum(Ret*Ret,axis=1)/nPoints #  mean value over time
    Phis[0,i] = tlist[cp.where(cp.max(Ret[0]) == Ret[0])[0][0]]*2*cp.pi
    Phis[1,i] = tlist[cp.where(cp.max(Ret[1]) == Ret[1])[0][0]]*2*cp.pi
    Phis[2,i] = tlist[cp.where(cp.max(Ret[2]) == Ret[2])[0][0]]*2*cp.pi



100%|██████████| 100/100 [00:00<00:00, 528.36it/s]

0.00020956993103027344
0.00010538101196289062
9.274482727050781e-05
9.250640869140625e-05
9.107589721679688e-05
9.298324584960938e-05
9.059906005859375e-05
9.131431579589844e-05
0.0001246929168701172
0.0001125335693359375
0.00010085105895996094
9.34600830078125e-05
9.226799011230469e-05
9.1552734375e-05
9.5367431640625e-05
0.0001468658447265625
0.000125885009765625
9.369850158691406e-05
0.0001690387725830078
9.894371032714844e-05
9.489059448242188e-05
9.298324584960938e-05
9.989738464355469e-05
9.918212890625e-05
9.870529174804688e-05
0.0001010894775390625
9.942054748535156e-05
9.870529174804688e-05
9.965896606445312e-05
9.846687316894531e-05
0.00010085105895996094
9.918212890625e-05
9.989738464355469e-05
9.775161743164062e-05
9.894371032714844e-05
0.00010180473327636719
9.870529174804688e-05
9.775161743164062e-05
9.942054748535156e-05
9.894371032714844e-05
0.00010180473327636719
9.846687316894531e-05
9.989738464355469e-05
9.965896606445312e-05
9.751319885253906e-05
0.00010013580322265

In [ ]:
# Data processing
# Fit with gamma functions
y=Amps[1,:]
y=y/np.mean(y)
y=np.sqrt(y)


plt.figure(figsize=(6,4))
# Histogram
Pr,bins,fig=plt.hist(y,30,density=True,label="Marginal Distribution of $A_{0}$",color="steelblue");
midbins = np.diff(bins)/2.+bins[:-1]

# curve fit
a,loc,scale = ss.gamma.fit(y,floc=0)
Pr_fit = ss.gamma.pdf(x=midbins,a=a,loc=loc,scale=scale)

plt.plot(midbins,Pr_fit,'black',linewidth=6,alpha=0.6,label='Gamma Fit')
plt.annotate(s="a = %.3f\n  shape = %.3f"%(a,scale),
             horizontalalignment="right",
         bbox=dict(boxstyle="round",fc="w"),xy=(0.98,0.5),xycoords='axes fraction',size=10)
plt.xlabel("Normalized $A_0$")
plt.ylabel("$Pr(A_0)$")
plt.legend()
#plt.savefig('GammaFit.pdf')

In [ ]:
# Check the linear combination of A_i
# Ak=np.sqrt((Ax**2+Ay**2+2*Ax*Ay*np.sin(Phix+Phiy))/2.)

In [ ]:
'''
Check the space coherence
'''
vp=1e-3
log10m = -22
m=10**log10m

t_corr = 2*np.pi/m
l_dB = 2*np.pi/m/vp
#ev2kpc = 3.0857e19 /1.973e-7


nPoints = 20

tx = np.zeros((4,nPoints))
tlist= np.linspace(0,1,nPoints+1)
tx[0,:] = tlist[:-1]*t_corr

ty = np.zeros((4,nPoints))
ty[0,:] = tlist[:-1]*t_corr
ty[1,:] = np.repeat(l_dB,nPoints)*2.56

nParticles = 1000
output=[]

nSimulation= 10000
Amps_x = np.ndarray((3,nSimulation))
Phis_x = np.ndarray((3,nSimulation))
Amps_y = np.ndarray((3,nSimulation))
Phis_y = np.ndarray((3,nSimulation))

'''
Main:
'''
for i in tqdm(range(nSimulation)):
    v,a,p=vec_particle_gen(vp,nParticles)

    # Physical values   
    m = 10**log10m # eV
    K = m*v

    # From K to omega
    omega = np.ndarray((1,nParticles))
    omega[0,:] = np.sqrt(np.sum(K*K,axis=0)+m**2)

    #Retx(3,nPoints)

    Ret_x = vector_random_walk(tx,omega,K,a,p)
    Ret_y = vector_random_walk(ty,omega,K,a,p)
    
    #results(3,nSimulation)
    Amps_x[:,i] = 2 * np.sum(Ret_x*Ret_x,axis=1)/nPoints #  mean value over time
    Phis_x[0,i] = tlist[np.where(np.max(Ret_x[0]) == Ret_x[0])[0][0]]*2*np.pi
    Phis_x[1,i] = tlist[np.where(np.max(Ret_x[1]) == Ret_x[1])[0][0]]*2*np.pi
    Phis_x[2,i] = tlist[np.where(np.max(Ret_x[2]) == Ret_x[2])[0][0]]*2*np.pi
    
    Amps_y[:,i] = 2 * np.sum(Ret_y*Ret_y,axis=1)/nPoints #  mean value over time
    Phis_y[0,i] = tlist[np.where(np.max(Ret_y[0]) == Ret_y[0])[0][0]]*2*np.pi
    Phis_y[1,i] = tlist[np.where(np.max(Ret_y[1]) == Ret_y[1])[0][0]]*2*np.pi
    Phis_y[2,i] = tlist[np.where(np.max(Ret_y[2]) == Ret_y[2])[0][0]]*2*np.pi


In [ ]:
x2 = Amps_x[1,:]
y2 = Phis_x[2,:]

x = np.sqrt(x2)
y = np.sqrt(y2)

m_x2 = np.mean(x2)
m_x = np.mean(x)
m_y2 = np.mean(y2)
m_y = np.mean(y)

sig_x = np.sqrt(m_x2-m_x**2)
sig_y = np.sqrt(m_y2-m_y**2)

result=np.mean((x-m_x)*(y-m_y))/sig_x/sig_y
print("%.3f"%(result))


In [ ]:
corr = np.ndarray((10,12))
Dis=np.array([0.0001, 0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56])
corr[0,:] = np.array([1.000,1.000,0.008,0.001,1.000,1.000,-0.006,0.004,0.014,0.015,-0.003,0.007])
corr[1,:] = np.array([0.998,0.998,0.013,-0.015,0.938,0.948,-0.018,0.001,-0.010,-0.002,0.001,-0.007])
corr[2,:] = np.array([0.991,0.991,0.005,-0.020,0.895,0.911,0.001,-0.003,0.000,0.022,-0.013,0.009])
corr[3,:] = np.array([0.964,0.965,0.009,-0.011,0.824,0.810,0.002,-0.008,0.021,-0.004,-0.016,0.007])
corr[4,:] = np.array([0.866,0.869,0.016,-0.010,0.658,0.672,-0.015,-0.019,0.001,-0.022,-0.020,0.018])
corr[5,:] = np.array([0.569,0.579,-0.006,0.006,0.446,0.438,0.005,0.005,0.023,-0.010,-0.008,-0.002])
corr[6,:] = np.array([0.126,0.125,-0.002,0.015,0.149,0.154,0.010,-0.016,0.009,0.004,0.006,0.001])
corr[7,:] = np.array([-0.008,0.004,-0.001,-0.000,0.017,0.023,-0.001,-0.003,0.010,-0.016,0.005,0.008])
corr[8,:] = np.array([-0.007,0.006,0.001,-0.014,-0.004,0.013,0.024,0.007,0.006,-0.001,-0.012,-0.001])
corr[9,:] = np.array([0.011,-0.003,-0.013,0.006,-0.003,0.007,0.008,0.011,-0.007,0.021,-0.020,-0.003])
plt.semilogx(Dis,corr);

In [ ]:
# Correlation Test-1, on amplitudes if certain direction 



valsx=np.array([0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56])
y1=[0.9999404204236939, 0.9997286376075016, 1.000073408701364,
 0.9999029418075727, 0.9998593391841898, 0.9993517781991963,
 0.9999465206974555, 0.9997576026360162, 0.9997438530657218]
y2=[0.9410427166670868, 0.939193879567938, 0.8845677195484672,
 0.7030079033709721, 0.48367021627090495, 0.4436117219154799,
 0.5052972335342074, 0.4743389963528783, 0.4673111944432566]
y100=[0.9810734888042447, 0.9408280718582631, 0.7925041174433759,
 0.3779713953921857, 0.015635133909090475, -0.048319716596257015,
 -0.047782568367157295, -0.034522005224365514, -0.027387532520900416]
y1000=[1.0012748295717215, 0.9706890421176969, 0.8175871118817806,
 0.46207917938094806, 0.053702680841951074, 0.015464042416849157,
 -0.00047152232801450633, -0.015587436498420686, 0.021327256909953848]




plt.semilogx(valsx,y1,'--')
plt.semilogx(valsx,y2)
plt.semilogx(valsx,y100)
plt.semilogx(valsx,y1000)

In [ ]:
# Amplitude as the function on N 

sqrtN=np.array(range(100)) 
rmsA=np.array([ 0.        ,  0.79846634,  1.61470283,  2.45912006,  3.2960502 ,
        4.10531182,  5.00936223,  5.65628151,  6.3610982 ,  7.22432485,
        8.18869485,  9.03596424,  9.89567429, 10.64053398, 11.64746189,
       11.94587072, 13.09847378, 13.79092467, 14.75556847, 15.46454741,
       16.52603156, 16.41034475, 18.03788818, 18.90438091, 19.88455712,
       20.67653797, 21.15476708, 21.74219317, 23.02839755, 23.94460244,
       24.5135315 , 25.62289196, 26.29387735, 26.60596482, 28.03526087,
       29.06469686, 29.13845635, 30.22109512, 29.74143123, 32.27197029,
       32.82808096, 33.09821604, 34.68934578, 35.6393422 , 36.43995517,
       36.91823923, 36.90650459, 38.42189739, 39.40285861, 40.08102474,
       40.9896192 , 42.48226713, 43.43985562, 44.51905154, 42.98135117,
       44.77947688, 45.86301141, 46.98701022, 46.67341974, 47.0655024 ,
       48.6029759 , 49.74842101, 51.54711642, 51.17306504, 53.33317334,
       53.1395299 , 54.64647966, 55.35221691, 56.6030676 , 56.45643468,
       56.49861522, 58.41489342, 59.35357932, 59.36948378, 60.31340694,
       61.45030096, 62.39700416, 62.3806855 , 64.7663445 , 64.61164241,
       65.74092418, 65.14686267, 66.73730258, 68.55993408, 68.8997811 ,
       69.96596792, 69.41275499, 72.39012902, 73.38264649, 73.58243626,
       72.77080659, 74.78611553, 74.98845046, 76.61442859, 78.7100685 ,
       76.14114011, 78.34209638, 78.63969245, 80.9677009 , 81.48174609])
meanA=np.array([ 0.        ,  0.62075403,  1.28704153,  1.96499303,  2.63348202,
        3.26266941,  3.9939252 ,  4.51085347,  5.06498872,  5.76844449,
        6.50449084,  7.20356297,  7.95839795,  8.5058701 ,  9.2555877 ,
        9.55251364, 10.43833793, 11.14392604, 11.62711766, 12.32752206,
       13.20932638, 13.14560063, 14.38737839, 15.02930709, 15.83167965,
       16.60510042, 16.81611099, 17.3031717 , 18.42177316, 19.08763107,
       19.47287412, 20.53271363, 20.91497086, 21.31568612, 22.4999253 ,
       23.41616609, 23.40032242, 24.02719153, 23.76023575, 25.66935356,
       26.07803993, 26.23784541, 27.60667146, 28.36006271, 29.14530548,
       29.66343084, 29.53712944, 30.67012787, 31.2958451 , 31.72407652,
       32.66972443, 33.74169982, 34.85269115, 35.65371956, 34.16837476,
       36.01088549, 36.76882923, 37.67857615, 37.39372945, 37.309618  ,
       38.97601965, 39.7727888 , 41.24697693, 40.32327808, 42.32301946,
       42.6533967 , 43.80413696, 44.47956186, 45.02282014, 45.1215823 ,
       45.16834647, 46.57989769, 47.7396148 , 47.25986284, 48.26060286,
       48.96629721, 49.86263602, 49.72826816, 51.69158344, 51.95797653,
       52.38111997, 52.45743527, 53.37935914, 55.11065483, 54.93432823,
       56.02484397, 55.78532571, 58.41918657, 58.02068968, 58.82864473,
       57.94621609, 60.20022989, 59.7776205 , 61.23974151, 62.60424592,
       60.37561381, 62.63116057, 62.59372356, 64.98788157, 65.38171219])

plt.plot(sqrtN,rmsA)
plt.plot(sqrtN,meanA)

In [ ]:
'''
Setups, change the parameters here!
'''
vp=1e-3
log10m = -22
m=10**log10m

t_corr = 2*np.pi/m
l_dB = 2*np.pi/m/vp
#ev2kpc = 3.0857e19 /1.973e-7


nTime = 10
nSpace = 100
nPoints = nTime * nSpace

tz = np.zeros((4,nPoints))
tz[0,:] = np.repeat(np.linspace(0,1,nTime)*t_corr,nSpace)
tz[1,:]= np.tile(ss.uniform.rvs(size=nSpace),nTime)*l_dB*10
tz[2,:]= np.tile(ss.uniform.rvs(size=nSpace),nTime)*l_dB*10
tz[3,:]= np.tile(ss.uniform.rvs(size=nSpace),nTime)*l_dB*10

nParticles = 1000

nSimulation= 10000
A02 = np.ndarray((nSpace,3,nSimulation))


for i in tqdm(range(nSimulation)):
    v,a,p=vec_particle_gen(vp,nParticles)

    # Physical values   
    m = 10**log10m # eV
    K = m*v

    # From K to omega
    omega = np.ndarray((1,nParticles))
    omega[0,:] = np.sqrt(np.sum(K*K,axis=0)+m**2)

    #Retx(3,nPoints)

    Ret = vector_random_walk(tz,omega,K,a,p)

    #results(3,nSimulation)
    for k in range(nSpace):
        A02[k,:,i] = 2* np.sum(Ret[:,k::nSpace]*Ret[:,k::nSpace],axis=1)/nTime #  mean value over time




In [ ]:
Pr,bins,fig=plt.hist(np.sqrt(A02[100,0,:]),50);

d = bins[1]-bins[0]
Pr = Pr*(1./nSimulation)*(1./d)
midbins = np.diff(bins)/2+bins[:-1]


#plt.hist(np.sqrt(A02[:,0,10]),20);

#np.mean(A02[:,0,:])/2

In [ ]:
plt.plot(midbins,Pr)
plt.plot(midbins,ss.maxwell.pdf(midbins/12.)/12.)

In [ ]:
'''
Get the statistical distribution of A**2 on uniformly distributed time-space coordinate.

Also the Monte-Carlo approximation of the integral.
'''
vp=1e-3
log10m = -22
m=10**log10m

t_corr = 2*np.pi/m
l_dB = 2*np.pi/m/vp
#ev2kpc = 3.0857e19 /1.973e-7


nPoints = 1000

tz = ss.uniform.rvs(size=(4,nPoints))
tz[0,:]=tz[0,:]*t_corr
tz[1:,:]=tz[1:,:]*l_dB*10

nParticles = 1000


nSimulation= 1000
outputC=np.ndarray((3,nSimulation))

'''
This part is slow:
'''
for i in tqdm(range(nSimulation)):
    v,a,p=vec_particle_gen(vp,nParticles)

    # Physical values   
    m = 10**log10m # eV
    K = m*v

    # From K to omega
    omega = np.ndarray((1,nParticles))
    omega[0,:] = np.sqrt(np.sum(K*K,axis=0)+m**2)

    #Ret(3,nPoints)

    Ret = vector_random_walk(tz,omega,K,a,p)

    outputC[:,i]=np.sum(Ret*Ret,axis=1)/nPoints




In [ ]:
plt.hist(outputC[0,:]);
np.sqrt(np.mean(outputC))